Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

import warnings
 
warnings.filterwarnings(action='ignore')

In [2]:
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


Data Processing

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shirleyw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/shirleyw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shirleyw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
fake_news = pd.read_csv("/Users/shirleyw/Desktop/PSTAT134/PSTAT134 Final Project/Fake.csv")
true_news = pd.read_csv("/Users/shirleyw/Desktop/PSTAT134/PSTAT134 Final Project/True.csv")

#add labels
fake_news["label"] = 0
true_news["label"] = 1

In [11]:
#combine fake news and true news into one dataset df
#Remove stop words
#Clean the reviews
df = pd.concat([fake_news, true_news], ignore_index = True)
df = df.drop(columns=["subject", "date"])
df['id'] = df.index

stop_words = set(stopwords.words('english'))

df.head()

,title,text,label,id
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0,2
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0,3
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0,4


In [12]:
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    return text


def remove_stop_words(text):
    #tokenize the text
    words = word_tokenize(text)
    #filter out the stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Join the filtered words back into a string
    return ' '.join(filtered_words)

# apply the function to the text column
df['cleaned_text'] = df['text'].apply(lambda text: ' '.join([word for word in text.split() if word.lower() not in stop_words]))
df['cleaned_text'] = df['cleaned_text'].apply(clean_text)


question: after cleanning the stop_words, the meaning of the sentences changed

In [13]:
print(df['cleaned_text'].iloc[0])
print(df['text'].iloc[0])

donald trump wish americans happy new year leave that instead give shout enemies haters dishonest fake news media former reality show star one job it country rapidly grows stronger smarter want wish friends supporters enemies haters even dishonest fake news media happy healthy new year president angry pants tweeted 2018 great year america country rapidly grows stronger smarter want wish friends supporters enemies haters even dishonest fake news media happy healthy new year 2018 great year america donald j trump realdonaldtrump december 31 2017trump tweet went welll expectwhat kind president sends new year greeting like despicable petty infantile gibberish trump lack decency even allow rise gutter long enough wish american citizens happy new year bishop talbert swan talbertswan december 31 2017no one likes calvin calvinstowell december 31 2017your impeachment would make 2018 great year america also accept regaining control congress miranda yaver mirandayaver december 31 2017do hear talk

In [14]:
df.head()

,title,text,label,id,cleaned_text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0,0,donald trump wish americans happy new year lea...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0,1,house intelligence committee chairman devin nu...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0,2,friday revealed former milwaukee sheriff david...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0,3,christmas day donald trump announced would bac...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0,4,pope francis used annual christmas day message...


TF-IDF

In [10]:
#libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline

In [15]:
# load and prepare the dataset
X = df['cleaned_text']
Y = df['label']

In [16]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=134)

In [17]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data (no fitting on test data)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [18]:
# Initialize the SVM classifier
svm_model = SVC(kernel='linear')

# Fit the SVM model on the TF-IDF transformed training data
svm_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [19]:
# Predict on the test set
y_pred = svm_model.predict(X_test_tfidf)

# Print the classification report
print(classification_report(y_test, y_pred))

# Print accuracy
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5954
           1       0.99      1.00      1.00      5271

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

Accuracy: 0.9957238307349666
